In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
## Loading and preparing the data

# Create mappings for each categorical data
OUTPUT_MAPPING = {0: "ham", 1: "spam"}

# Replace categorical data in the dataset in place & load the training and testing datasets
train_dataset = pd.read_csv(train_file_path, sep='\t', header=None)
test_dataset = pd.read_csv(test_file_path, sep='\t', header=None)

train_dataset[0].replace(
    ['ham', 'spam'],
    [0, 1],
    inplace=True
)
test_dataset[0].replace(
    ['ham', 'spam'],
    [0, 1],
    inplace=True
)

train_labels = train_dataset.pop(0)
test_labels = test_dataset.pop(0)

In [ ]:
## Text preprocessing

# Tokenize the text data to sequences of integers
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_dataset[1])

# Convert the text to sequences (lists of integers)
train_sequences = tokenizer.texts_to_sequences(train_dataset[1])
test_sequences = tokenizer.texts_to_sequences(test_dataset[1])

# Pad sequences to ensure they all have the same length of 100
train_sequences_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100)
test_sequences_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100)

# Convert labels to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
# Creating the model
vocab_size = len(tokenizer.word_index) + 1

model = keras.models.Sequential()
model.add(keras.layers.Embedding(input_dim=vocab_size, output_dim=32, input_length=100))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.summary()

# Training the model
history = model.fit(
    train_sequences_padded,
    train_labels,
    epochs=8,
    batch_size=32,
    validation_data=(
      test_sequences_padded,
      test_labels
    ),
    verbose=1
)

test_loss, test_accuracy = model.evaluate(test_sequences_padded, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Plot accuracy
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  encoded_text = tokenizer.texts_to_sequences([pred_text])
  pred_text_padded = keras.preprocessing.sequence.pad_sequences(encoded_text, maxlen=100)

  prediction = model.predict(pred_text_padded)

  probability = prediction.flatten()[0]
  label = OUTPUT_MAPPING[int(round(probability))]

  return [probability, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
